In [1]:
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  bag = {}  # stores all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]  # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1

    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1

  return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [2]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

print("Positive:", pos_bag)
print("Negative:", neg_bag)

Positive: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1}
Negative: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 21: 1, 18: 1, 19: 1, 20: 1, 17: 1}


In [3]:
vocab = {}
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ")
  encoding = []

  for word in words:
    if word in vocab:
      code = vocab[word]
      encoding.append(code)
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1

  return encoding

text = "this is a test to see if this test will work is is test a a"
encoding = one_hot_encoding(text)
print(encoding)
print(vocab)

[1, 2, 3, 4, 5, 6, 7, 1, 4, 8, 9, 2, 2, 4, 3, 3]
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [4]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_encode = one_hot_encoding(positive_review)
neg_encode = one_hot_encoding(negative_review)

print("Positive:", pos_encode)
print("Negative:", neg_encode)

Positive: [10, 11, 12, 13, 14, 15, 5, 16, 17, 18, 19, 14, 20, 21]
Negative: [10, 11, 12, 13, 14, 15, 5, 16, 21, 18, 19, 14, 20, 17]


# Sentiment Analysis

In [5]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
17464789/17464789 [==============================] - 0s 0us/step


In [6]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 48s 71ms/step - loss: 0.4471 - acc: 0.7807 - val_loss: 0.3054 - val_acc: 0.8728
Epoch 2/10
625/625 [==============================] - 17s 27ms/step - loss: 0.2576 - acc: 0.9008 - val_loss: 0.3006 - val_acc: 0.8822
Epoch 3/10
625/625 [==============================] - 12s 20ms/step - loss: 0.2024 - acc: 0.9253 - val_loss: 0.3566 - val_acc: 0.8578
Epoch 4/10
625/625 [==============================] - 11s 17ms/step - loss: 0.1642 - acc: 0.9436 - val_loss: 0.2914 - val_acc: 0.8770
Epoch 5/10
625/625 [==============================] - 9s 15ms/step - loss: 0.1405 - acc: 0.9521 - val_loss: 0.3340 - val_acc: 0.8536
Epoch 6/10
625/625 [==============================] - 10s 16ms/step - loss: 0.1178 - acc: 0.9617 - val_loss: 0.3222 - val_acc: 0.8766
Epoch 7/10
625/625 [==============================] - 9s 14ms/step - loss: 0.0989 - acc: 0.9685 - val_loss: 0.3477 - val_acc: 0.8750
Epoch 8/10
625/625 [==============================] - 8s 13ms/st

In [9]:
results = model.evaluate(test_data, test_labels)
print(results)

In [10]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


1641221/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [11]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [12]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

1/1 [==============================] - 1s 658ms/step
[0.935653]
1/1 [==============================] - 0s 29ms/step
[0.127334]


# RNN Play Generator

In [13]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [14]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [16]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [17]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [18]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [19]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [20]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [21]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [22]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [23]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [24]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [25]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [26]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_1 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5330241 (20.33 MB)
Trainable params: 5330241 (20.33 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [28]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-2.37431820e-03 -5.47135063e-03 -1.00572072e-02 ... -3.73502146e-03
    3.15275020e-03  2.63041817e-03]
  [-3.57317016e-03 -8.10095295e-03 -4.89645172e-03 ... -2.78984942e-03
    8.16932414e-03  2.32312409e-03]
  [-4.74681240e-03 -7.93812145e-03 -9.45764594e-03 ... -1.43070263e-03
    4.24548052e-04  4.70852759e-03]
  ...
  [ 5.04184607e-03 -5.29281143e-03  3.21232900e-03 ...  1.03200378e-03
    3.38081480e-03  1.03446189e-04]
  [ 2.44308892e-03 -9.71749984e-03  3.85338278e-03 ...  1.52525678e-03
    7.92136509e-03  2.57126335e-03]
  [ 4.56746016e-03 -1.10994447e-02  1.16132689e-03 ...  7.63508433e-05
    5.95606957e-03  6.65921252e-04]]

 [[-2.37431820e-03 -5.47135063e-03 -1.00572072e-02 ... -3.73502146e-03
    3.15275020e-03  2.63041817e-03]
  [ 5.31854108e-04  1.18309376e-03 -8.21532123e-03 ... -5.10108937e-03
    4.92013618e-03  5.18953335e-03]
  [-6.95144292e-04  2.09625205e-03 -3.43981315e-03 ... -5.94932353e-03
    3.71287554e-03  6.42467057e-04]
  ...
  [-5.763

In [29]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-2.3743182e-03 -5.4713506e-03 -1.0057207e-02 ... -3.7350215e-03
   3.1527502e-03  2.6304182e-03]
 [-3.5731702e-03 -8.1009530e-03 -4.8964517e-03 ... -2.7898494e-03
   8.1693241e-03  2.3231241e-03]
 [-4.7468124e-03 -7.9381214e-03 -9.4576459e-03 ... -1.4307026e-03
   4.2454805e-04  4.7085276e-03]
 ...
 [ 5.0418461e-03 -5.2928114e-03  3.2123290e-03 ...  1.0320038e-03
   3.3808148e-03  1.0344619e-04]
 [ 2.4430889e-03 -9.7174998e-03  3.8533828e-03 ...  1.5252568e-03
   7.9213651e-03  2.5712634e-03]
 [ 4.5674602e-03 -1.1099445e-02  1.1613269e-03 ...  7.6350843e-05
   5.9560696e-03  6.6592125e-04]], shape=(100, 65), dtype=float32)


In [30]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-0.00237432 -0.00547135 -0.01005721  0.00546232 -0.00102907 -0.00260884
  0.00152438 -0.00155765 -0.00083136 -0.00206626 -0.00303523  0.00109406
 -0.00016082 -0.00274255  0.00257152  0.00017511 -0.00190918  0.00812275
 -0.0033444   0.00202222  0.0037367   0.00269834  0.00210156 -0.001531
 -0.00187128  0.00277537  0.0032971   0.00079104 -0.00111711  0.00058686
  0.00133974  0.00200258 -0.00059453 -0.00291448  0.00191896 -0.00206861
  0.01194951  0.00021201  0.0026182   0.00166131 -0.00435438 -0.00317055
 -0.00498145  0.00802493 -0.00087644  0.00382206 -0.00218373 -0.00163302
 -0.00606679  0.00087754 -0.00132161  0.00327915  0.00333717  0.00012884
 -0.00413076  0.00216884  0.00942067  0.00116553  0.00023523 -0.00144204
  0.00237971 -0.00312414 -0.00373502  0.00315275  0.00263042], shape=(65,), dtype=float32)


In [31]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

'E?,qFXT:y:TD!;bRyLRu?GUGH!.A\ngfpWJ uAekO gT:PWS WUBWV!EUUomulKYpqXE&-QeYHBMO\nzxDIaxvGinP$kcbJdov,zBt'

In [32]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [33]:
model.compile(optimizer='adam', loss=loss)

In [34]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [35]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 16s 67ms/step - loss: 2.7925
Epoch 2/50
172/172 [==============================] - 14s 65ms/step - loss: 2.0907
Epoch 3/50
172/172 [==============================] - 13s 65ms/step - loss: 1.8224
Epoch 4/50
172/172 [==============================] - 13s 65ms/step - loss: 1.6622
Epoch 5/50
172/172 [==============================] - 13s 66ms/step - loss: 1.5605
Epoch 6/50
172/172 [==============================] - 14s 68ms/step - loss: 1.4914
Epoch 7/50
172/172 [==============================] - 13s 67ms/step - loss: 1.4414
Epoch 8/50
172/172 [==============================] - 13s 68ms/step - loss: 1.4038
Epoch 9/50
172/172 [==============================] - 13s 67ms/step - loss: 1.3722
Epoch 10/50
172/172 [==============================] - 14s 68ms/step - loss: 1.3441
Epoch 11/50
172/172 [==============================] - 13s 68ms/step - loss: 1.3202
Epoch 12/50
172/172 [==============================] - 14s 68ms/step - loss: 1.2975
E

In [36]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [37]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [39]:
checkpoint_num = 10
model.load_weights("./training_checkpoints/ckpt_" + str(checkpoint_num))
model.build(tf.TensorShape([1, None]))

In [40]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension

      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [42]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: apple
appleadon's require and die block--
To nothing more than Perca's sou to-done.
Farther, I prithee, leist upon it not oath.
I none of bold, then way to think
'Tis gollory, and quickly, who At Paulina
and read your vict be; pardo? O, to be quiet
And flet us to't:
Lords, thus; I am song corposion would not.

KING RICHARD III:
Sunnoc and hacts moctagnors. 'ha resoleity,
And where finds the
queen do we hear.

QUEEN MARGARET:
I will't know you were no more than zalack and blown,
Three wermiatedeaments, so finds another and I can a better breath.

KING RICHARD III:
Good lorrs,
But that I say, bring me bold,
To ase I in that; be arro; but they
Have sains the gave by a death; you re jest in auit Henry's blood
With from me with ink beginst years within Edward. Now, lords, I know you, sir, thou
prayers, di
